# Dataset Size Comparison

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [ ]:
def size_barplot(data_type: str, input_folder: str, output_folder: str, metric: str) -> None:
   
    '''
    Barplot showing the average perfomances across the experiments of meme, streme and ls-gkm when
    using different dataset sizes.
    
    data_type: one of the following ["dnase-shuffle", "shuffle-dnase", "dnase", "shuffle"]
    input_folder: folder containing all the data regarding dnase, shuffle, dnase-shuffle and shuffle-dnase
    output_folder: where the barplot will be created, structured based on the data_type 
    metric: one of the following ["auroc","auprc", "f1score"]
    
    It creates the barplot regarding the selected data_type and metric, retrieving the data from the 
    "{input_folder}/{data_type}/dataset-size-comparison" folder and saving it in the 
    "{output_folder}/{data_type}/dataset_size_comparison_barplot" barplot.
    
    '''
    
    possible_data_types = ["dnase-shuffle", "shuffle-dnase", "dnase", "shuffle"]
    if data_type not in possible_data_types:
        raise ValueError(f"data_type must be one of the following: {possible_data_types}")
    
    possible_metrics = ["auroc","auprc", "f1score"]
    if metric not in possible_metrics:
        raise ValueError(f"metric must be one of the following: {possible_metrics}")
    
    # Set the input and output folder
    data_folder = f"{input_folder}/{data_type}/dataset-size-comparison"
    output_folder = f"{output_folder}/{data_type}/dataset_size_comparison_barplot"
    
    # Create the output folder(s)
    os.makedirs(output_folder, exist_ok=True)
    
    # Sizes and tools that will be compared 
    size_list = [500, 1000, 2000, 5000, 10000, 'full']
    tools = ["meme","streme","svm"]
    
    # Load the data
    data_sizes = {}
    for size in size_list:
        file_path = f"{data_folder}/summary_table_size_{size}.tsv"
        data_sizes[size] = pd.read_csv(file_path, sep="\t")
    
    # Retrieve the data refering to "metric" only
    
    metric_col_name = "F1" if metric == "f1score" else metric.upper()
    metric_data = {}
    for size, df in data_sizes.items():
        metric_cols = [col for col in df.columns if col.startswith(metric_col_name) or col.startswith("EXP")]
        metric_data[size] = df[metric_cols]

    # Compute for each tool and size the mean and standard deviation

    metric_mean_dicts = {}
    metric_std_dicts = {}
    for size, df in metric_data.items():
        means = [df[col].mean().round(3) for col in df.columns[1:]]
        stds = [df[col].std().round(3) for col in df.columns[1:]]
        tool_means = dict(zip([col.split("_")[1] for col in df.columns[1:]], means))
        tool_stds = dict(zip([col.split("_")[1] for col in df.columns[1:]], stds))

        metric_mean_dicts[size] = {tool: tool_means[tool] for tool in tools}
        metric_std_dicts[size] = {tool: tool_stds[tool] for tool in tools}
    
    # Plot

    barWidth = 0.3
    fig, ax = plt.subplots(figsize=(20, 15), dpi=300)

    # Bar positions
    br_positions = {size: [] for size in size_list}
    br1 = np.arange(len(size_list))

    for index, size in enumerate(size_list):
        br_positions[size] = [br1[index] + (barWidth + 0.00) * x for x in range(len(tools))]

    colors = ["#f7b32bff", "#d52a39ff", "#366fdaff"]  # meme, streme, ls-gkm
    for index, size in enumerate(size_list):
        plt.bar(
            br_positions[size],
            metric_mean_dicts[size].values(),
            color=colors,
            width=barWidth,
            edgecolor="grey",
            label=["meme", "streme", "ls-gkm"] if index == 0 else None,
            zorder=3
        )
        for i, (tool, mean) in enumerate(metric_mean_dicts[size].items()):
            std = metric_std_dicts[size][tool]
            plt.text(
                br_positions[size][i],
                mean + 0.055,
                f"std: \n{std}",
                horizontalalignment="center",
                fontsize=15
            )
            plt.text(
                br_positions[size][i],
                mean + 0.01,
                f"mean: \n{mean:.3f}",
                horizontalalignment="center",
                fontsize=15
            )

    # Axis configuration
    plt.xticks([r + barWidth for r in range(len(size_list))], size_list, fontsize=15)
    plt.yticks(fontsize=15)
    plt.legend(loc=2, prop={'size': 18})
    plt.grid(axis="y", zorder=0, alpha=0.3)
    ylabel = "F1-Score" if metric == "f1score" else metric.upper()
    plt.ylabel(ylabel, fontsize=30)
    plt.xlabel("Dataset size", fontsize=30)
    plt.title(f"{data_type} {ylabel} mean of tools", fontsize=30)
    ax.set_ylim([0, 1.1])

    plt.savefig(f"{output_folder}/{ylabel}_dataset_size.svg")
    plt.savefig(f"{output_folder}/{ylabel}_dataset_size.png")
    plt.show()

In [ ]:
size_barplot(data_type = "", input_folder = "", output_folder = "", metric = "")